In [1]:
#1> load the environment + define the path
%reload_ext autoreload
%autoreload 2

#make data file
print( "Takes <30s. However, it might take >1 mins or so if network is busy...")
import glob
import os, glob, shutil
import pandas as pd
from datetime import datetime
import numpy as np
import main_funcs as mfun
import utils_funcs as utils # utils is from Vape - catcher file: 
import matplotlib.pyplot as plt
import pickle
import sys
from LakLabAnalysis.Utility.extract_paq_events import extract_paq_data_frame
from tifffile import TiffFile, imread, imwrite
import imageio.v2 as imageio

# set matlab API
import matlab.engine
eng = matlab.engine.start_matlab()
print('Matlab engine is set correctly.')



Takes <30s. However, it might take >1 mins or so if network is busy...
Matlab engine is set correctly.


In [13]:
# Get the list of recordings
animalList= ['MAT058',]#'MAT066','MAT067']
#'HA011','HAT009',
info = mfun.analysis(animalList=animalList)
# display the detected session
print( "Total Session before exclsuion criterias: " +  str(info.recordingList .shape[0]))

###### Lets clean the short and bad performing sessions
# Cleaning step 1: Eliminate short sessions - exclude less than 10 mins
info.recordingList = info.recordingList[info.recordingList['duration'] > 10] # >10 mins
# Cleaning step 2: Eliminate bad performing sessions
info.recordingList = info.recordingList[info.recordingList['performance'] > 60].reset_index(drop=True) # >50% performance

###### Lets See all table
# display the detected session
print( "Total Session after exclsuion criterias: " +  str(info.recordingList .shape[0]))
print(info.recordingList.to_string())



Env: decMaking
Computer: Huriye Windows
Total Session before exclsuion criterias: 24
Total Session after exclsuion criterias: 20
   animalID recordingDate recordingID          sessionName   twoP                  path                        imagingTiffFileNames                                   sessionNameWithPath            blockName                        experimentDefinition   duration    rigName  performance                                    stimType                                                                     analysispathname            filepathname
0    MAT058    2025-10-09           1  2025-10-09_1_MAT058  False  Z:\MAT058\2025-10-09                                        None  Z:\MAT058\2025-10-09\1\2025-10-09_1_MAT058_Block.mat  2025-10-09_1_MAT058  Grating2AFC_variableStimSize_variabledelay  32.543750  win-al021    66.111111  Grating2AFC_variableStimSize_variabledelay  C:\Users\Huriye\Documents\code\sideBiasLateralisation\analysis\2025-10-09_MAT058_1\  Z:\MAT058\2025-1

In [10]:
# Lets set the matlab correctly
print('Matlab version: ', eng.version())
eng.restoredefaultpath(nargout=0)
eng.rehash('toolboxcache', nargout=0)
print('MATLAB path restored to default.')
eng.addpath(eng.genpath(info.rootPath), nargout=0)
print(f'Added {info.rootPath} and all its subfolders to the MATLAB path.')
eng.savepath(nargout=0)

Matlab version:  25.1.0.2973910 (R2025a) Update 1
MATLAB path restored to default.
Added C:\Users\Huriye\Documents\code\sideBiasLateralisation\ and all its subfolders to the MATLAB path.


In [14]:
# For checking performance over time
# Lets plot sessions in animal folders - 
animalList = np.unique(info.recordingList['animalID'])
for animal in animalList:
    print(f"Processing animal: {animal}")
    animal_sessions = info.recordingList.loc[info.recordingList['animalID'] == animal, :].reset_index(drop=True)
    # each animal one folder
    save_path = os.path.join(info.analysisPath, animal)
    save_path_session = os.path.join(save_path, 'session')
    #create if not exist ( in one line)
    os.makedirs(save_path, exist_ok=True) if not os.path.exists(save_path) else None
    os.makedirs(save_path_session, exist_ok=True) if not os.path.exists(save_path_session) else None
    csv_file = os.path.join(save_path, f"{animal}_perf.csv")
    if os.path.exists(csv_file):
        pd_beh = pd.read_csv(csv_file) 
    else:
        pd_beh = pd.DataFrame(columns=['session', 'performance', 'twoP']) 

    # Plot each session 
    for ind, row in animal_sessions.iterrows():
        
        session = row['sessionName']     # adjust column name if different
        twoP = row['twoP']
        save_file = os.path.join(save_path_session,f"{session}_beh.png")
        if not os.path.exists(save_file):
            try:
                print(session)
                perf = eng.visualiseTrainingGrating2AFC(session, 'Grating2AFC_noTimeline', [], nargout=1)
                print(perf)
                fig = eng.gcf()
                eng.saveas(fig, save_file, nargout=0)
                eng.close('all', nargout=0) 
                
            except Exception as e:
                print(f"{session} ---> Error in Matlab getBehaData function - maybe wrong profile?! ")
                perf = np.nan

            # append a new row to pd_beh
            pd_beh.loc[len(pd_beh)] = {
                "session": session[5:10],
                "twoP": twoP,
                "performance": perf
            }

            # update performance in original dataframe
            df = info.recordingList
            mask = (df['animalID'] == animal) & (df['sessionName'] == session)
            info.recordingList.loc[mask, 'performance'] = perf

    # save pd_beh
    pd_beh.to_csv(csv_file, index=False)

    # Plot summary plot session
    perf_save_file = os.path.join(save_path, f"0behSummary_{animal}.png")
    plt.plot(pd_beh['session'], pd_beh['performance'], label='Performance', color='black')
    # Mark twoP
    mask_twoP = pd_beh['twoP'] == True
    plt.scatter(
        pd_beh.loc[mask_twoP, 'session'], 
        pd_beh.loc[mask_twoP, 'performance'],
        facecolors='none', edgecolors='red', s=50, linewidths=2, label='TwoP'
    )

    plt.title(f"Summary Performance Plot - {animal}")
    plt.ylim(0, 100)  # Set y-axis limits
    plt.axhline(y=50, color='grey', linestyle='--') #add a dash line at 50
    plt.xlabel("Session")
    # lets rotate x labels
    plt.xticks(rotation=60, fontsize=8)
    plt.ylabel("Performance")
    plt.savefig(perf_save_file)
    plt.close()
# End of session loop
print('Finished processing all sessions. Saved:' + info.analysisPath)

Processing animal: MAT058
2025-10-13_1_MAT058
76.70454545454545
2025-10-14_1_MAT058
82.3045267489712
2025-10-15_1_MAT058
92.6829268292683
2025-10-16_1_MAT058
76.16279069767442
2025-10-17_1_MAT058
70.46632124352331
2025-10-21_1_MAT058
69.53125
2025-10-22_1_MAT058
85.84474885844749
2025-10-23_2_MAT058
87.67772511848341
2025-10-24_1_MAT058
77.07509881422925
2025-10-27_1_MAT058
84.15841584158416
2025-10-28_1_MAT058
74.609375
2025-10-29_1_MAT058
78.515625
2025-10-30_1_MAT058
65.18218623481782
2025-10-31_1_MAT058
67.19367588932806
2025-11-03_1_MAT058
76.98744769874477
2025-11-05_1_MAT058
72.37354085603113
2025-11-07_1_MAT058
84.23236514522821
2025-11-10_1_MAT058
88.12260536398468
Finished processing all sessions. Saved:C:\Users\Huriye\Documents\code\sideBiasLateralisation\analysis
